In [1]:
import torch
import math

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1660 Ti'

In [3]:
from ultralytics import YOLO

In [4]:
model=YOLO("G:/ml notebooks/Road_RUL/datasets/runs/detect/train3/weights/best.pt")

In [5]:
results=model.predict(source="G:/ml notebooks/Road_RUL/datasets/all_images/China_MotorBike_000950.jpg",save=True,save_txt=True)


image 1/1 G:\ml notebooks\Road_RUL\datasets\all_images\China_MotorBike_000950.jpg: 640x640 3 D40s, 13.0ms
Speed: 6.0ms preprocess, 13.0ms inference, 141.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3
1 label saved to runs\detect\predict3\labels


In [6]:
results[0]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'D00', 1: 'D40', 2: 'D10', 3: 'D20', 4: 'Repair', 5: 'Block crack', 6: 'D44', 7: 'D43', 8: 'D01', 9: 'D11', 10: 'D50', 11: 'D0w0'}
orig_img: array([[[160, 155, 152],
        [155, 150, 147],
        [148, 143, 140],
        ...,
        [127, 123, 122],
        [133, 129, 128],
        [130, 126, 125]],

       [[164, 159, 156],
        [153, 148, 145],
        [144, 139, 136],
        ...,
        [126, 122, 121],
        [128, 124, 123],
        [124, 120, 119]],

       [[163, 158, 155],
        [151, 146, 143],
        [146, 141, 138],
        ...,
        [128, 124, 123],
        [130, 126, 125],
        [126, 122, 121]],

       ...,

       [[134, 133, 135],
        [153, 152, 154],
        [157, 156, 158],
        ...,
        [143, 142, 146],
        [188, 187, 191],
        [223, 222, 226]],

       [[106, 105, 107],
        [161, 1

In [7]:
results[0].boxes

WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([[375.5027,  98.5197, 481.7326, 260.5540,   0.8525,   1.0000],
        [237.2420, 227.6635, 318.4546, 511.3420,   0.7767,   1.0000],
        [335.1393, 194.1595, 388.5824, 324.0883,   0.7674,   1.0000]], device='cuda:0')
cls: tensor([1., 1., 1.], device='cuda:0')
conf: tensor([0.8525, 0.7767, 0.7674], device='cuda:0')
data: tensor([[375.5027,  98.5197, 481.7326, 260.5540,   0.8525,   1.0000],
        [237.2420, 227.6635, 318.4546, 511.3420,   0.7767,   1.0000],
        [335.1393, 194.1595, 388.5824, 324.0883,   0.7674,   1.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (512, 512)
shape: torch.Size([3, 6])
xywh: tensor([[428.6176, 179.5368, 106.2300, 162.0344],
        [277.8483, 369.5028,  81.2126, 283.6785],
        [361.8609, 259.1238,  53.4431, 129.9288]], device='cuda:0')
xywhn: tensor([[0.8371, 0.3507, 0.2075, 0.3165],
        [0.5427, 0.7217, 0.1586, 0.5541],
        [0.7068, 0.5061, 0.1044, 0.

we need xyxy data but in pixels,not normalized

Since Our other model was unable to calculate RUL properly, After doing intense research into this topic and spending almost a month collecting and going through research papers and articles, I have found this approach quite simple to use and can easily find the RUL

Accuracy will be compromised for the sake of simplicity of this project, I will be mentioning what more can be done in Scope For Improvement section in Project Report.

In [8]:
def output_fetch(results):
    #I have just made one single codeblock to get our results
    # # Move the results to CPU
    results = [result.cpu() for result in results]
    lis=[]
    # Convert the results to numpy
    results = [result.numpy() for result in results]
    # Get the bounding boxes
    boxes = results[0].boxes
    for box in boxes:
        type_of_damage=results[0].names[box.cls[0]]
        dpi=96
        px_to_mm=25.4/dpi
        # Get the bounding box in the format [x1, y1, x2, y2]Top Left and Bottom Right
        bbox = box.xyxy[0]
    
        # Convert the bounding box coordinates to pixels
        image_width = results[0].orig_img.shape[1]*px_to_mm
        image_height = results[0].orig_img.shape[0]*px_to_mm
    
        x1 = bbox[0] * image_width
        y1 = bbox[1] * image_height
        x2 = bbox[2] * image_width
        y2 = bbox[3] * image_height
    
        # Calculate the width and height of the bounding box
        width = (x2 - x1)*px_to_mm
        height = (y2 - y1)*px_to_mm
    
        width=width*px_to_mm
        height=height*px_to_mm
    
        # Calculate the area covered by the bounding box
        area = width * height
        area=area*(px_to_mm**2)
    
        # Calculate the area taken up by the bounding box relative to the image
        image_area = image_width * image_height
    
        relative_area = (area / image_area)
        lis.append([type_of_damage,((width+height)/2),relative_area])
        
        
    return lis
        



## Distress Params
- Severity Of Distress-> Average of width and height is taken to get a rough estimate of length of the crack which points towards severity roughly.
 If this average :
 <500: Very Slight(1) ,
 <800: Slight(2), 
 <1200: Moderate(3)  
 <1500: Severe, 
 1500+:Very Severe(5)

- Density Of Distress-> Relative area of the bounding box and the total area of the image itself,can be improved with a segmentation model
If relative area:
1-3 : Few(1)
4-6 : Intermitent(2)
7-10 : Frequent(3)
11-14 : Extensive(4)
15+ : Throughout(5)

## Simplifying the types of damages
-Lets condense these down to Just Cracks and Others
Withing Cracks, we have Linear and Alligator

>D00,D01,D10,D11 are Linear Crack
D20 is Alligator Crack
D40,D43,D44 and etc are Others

- Severity can be kind of calculated by the width or height to show how big it is and the density is the relative area, we add these and then multiply it with the Wi value, add them up to get PCI, from PCI, there is a direct relation or a general ranges where we can determine RUL. This is done in modify() function

In [9]:
results[0].names

{0: 'D00',
 1: 'D40',
 2: 'D10',
 3: 'D20',
 4: 'Repair',
 5: 'Block crack',
 6: 'D44',
 7: 'D43',
 8: 'D01',
 9: 'D11',
 10: 'D50',
 11: 'D0w0'}

In [10]:
def modify(lis):
    finals=[]
    for dmges in lis:
    #Get crack_type
        if dmges[0] in ['D00','D01','D10','D11']:
            crack_type="linear"
        elif dmges[0]=='D20':
            crack_type="alligator"
        else:
            crack_type="other"
         
        #Get severity rating    
        if dmges[1]<=500:
            severity_rating=1
        elif dmges[1]>500 and dmges[1]<=800:
            severity_rating=2
        elif dmges[1]>800 and dmges[1]<=1200:
            severity_rating=3
        elif dmges[1]>1200 and dmges[1]<=1500:
            severity_rating=4
        else:
            severity_rating=5
        
        #Get density_rating
        if dmges[2]<=3:
            density_rating=1
        elif dmges[2]>3 and dmges[2]<=6:
            density_rating=2
        elif dmges[2]>6 and dmges[2]<=10:
            density_rating=3
        elif dmges[2]>10 and dmges[2]<=14:
            density_rating=4
        else:
            density_rating=5
        
        finals.append([crack_type,severity_rating,density_rating])
    return finals
        
    
    

In [11]:
def PCI_Calc(finals):
  dmi=0
  """Calculates the Pavement Condition Index (PCI) of a road section, condensed to linear, alligator, and other cracks, where the weights are reduced by a smaller factor.
  Args:
  A list of lists which contains the following for each damage detected:
    severity rating: The severity of the damage, from 1 to 5.
    crack_type: The type of crack, as a string, either "linear", "alligator", or "other".
    density_rating: The density rating of the damage, from 1 to 5.

  Returns:
    The PCI value, a float between 0 and 100.
  """
  for f in finals:
    # Get the weight for the crack type.
    weight = {
      "linear": 2.7,
      "alligator": 2.2,
      "other": 1.5,
      }.get(f[0])
    # Calculate the Distress Manifestation Index (DMI).
    dmi+=(f[1]+f[2])*weight


  pci=100-dmi
  return pci


In [28]:
results=model.predict(source="G:/ml notebooks/Road_RUL/datasets/all_images/Czech_001876.jpg",save=True,save_txt=False)
pci_value=PCI_Calc(modify(output_fetch(results)))


image 1/1 G:\ml notebooks\Road_RUL\datasets\all_images\Czech_001876.jpg: 640x640 1 D40, 182.4ms
Speed: 8.0ms preprocess, 182.4ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3


In [29]:
if pci_value>70:
    quality="Good and does not need improvement, Maintenance is not urgent"
elif pci_value<70 and pci_value>55:
    quality="Fair but needs maintenace soon,scope for improvement present"
else:
    quality="Need Maintenance, Poor Quality and less service life"

Now lets find Remaining Service Life

In [30]:
import numpy as np

In [31]:
RUL=4.1872*(np.log(pci_value))-14.117
print(RUL)

5.038229809189364


Highest value possible is when PCI is perfect i.e 100,then we get RUL as 5.2 which basically means no road should go without maintenance for more than 5 years, although the road might be useful, according to general predictive maintenance procedures, the results are not too far off.